In [ ]:
!pip install kaggle
!pip install tensorflow
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Pre-trained modeller
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import MobileNet

import tensorflow as tf

"""## 2. Veri Seti Yükleme (Face Mask Detection Dataset)
Kaggle’dan veri setinin indirilmesi
"""

from google.colab import files
!mkdir ~/.kaggle

# Veri setini indirme
!kaggle datasets download -d omkargurav/face-mask-dataset

# İndirdiğiniz zip dosyasını açma
!unzip face-mask-dataset.zip -d face_mask_dataset

"""## 3. Önceden Eğitilmiş (Pre-trained) Modellerin Hazırlanması
Projede DenseNet121’i kullanarak bir model oluşturulmuştur.
"""

IMAGE_SIZE = (224, 224)

base_model = DenseNet121(weights='imagenet',
                         include_top=False,
                         input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Base model'in son katmanından çıkan aktivasyon haritalarının alınması
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)

# Ek dense katmanları
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Çıkış katmanı (2 sınıf: With Mask (0), Without Mask (1))
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

"""## 4. Veri Setinin Eğitim/Validasyon ve Test Olarak Ayrılması

Toplam verinin %20’sini test set olarak ayıracağız. Geri kalan %80’i ise eğitim+validasyon şeklinde kullanacağız.

#### Klasör Bazlı Ayırma:
"""

import shutil
import os
from glob import glob

data_dir = 'face_mask_dataset/data'
classes = ['with_mask', 'without_mask']

# Hedef klasörlerin oluşturulması
os.makedirs('dataset/train/with_mask', exist_ok=True)
os.makedirs('dataset/train/without_mask', exist_ok=True)
os.makedirs('dataset/val/with_mask', exist_ok=True)
os.makedirs('dataset/val/without_mask', exist_ok=True)
os.makedirs('dataset/test/with_mask', exist_ok=True)
os.makedirs('dataset/test/without_mask', exist_ok=True)

test_size = 0.20  # %20 test
val_size = 0.20   # Train+Val'in içinde %20 validation

for cls in classes:
    all_images = glob(os.path.join(data_dir, cls, '*.png')) + \
                 glob(os.path.join(data_dir, cls, '*.jpg')) + \
                 glob(os.path.join(data_dir, cls, '*.jpeg'))

    # 1) Test ayırma
    train_val_files, test_files = train_test_split(all_images,
                                                   test_size=test_size,
                                                   random_state=42)

    # 2) Train ve Validation ayırma (train_val_files içerisinden %20'si validation)
    train_files, val_files = train_test_split(train_val_files,
                                              test_size=val_size,
                                              random_state=42)

    # Dosyaların ilgili klasörlere kopyalanması
    for f in train_files:
        shutil.copy(f, f'dataset/train/{cls}')
    for f in val_files:
        shutil.copy(f, f'dataset/val/{cls}')
    for f in test_files:
        shutil.copy(f, f'dataset/test/{cls}')

"""## 5. Eğitim/Validasyon Setinin Bölünmesi ve Modelin Eğitilmesi

Artık train, val klasörlerimizde eğitim ve doğrulama verimiz hazır. Bu veriler üzerinde data augmentation (veri arttırımı) uygulayacağız ve modelimizi eğiteceğiz.

#### ImageDataGenerator ile Veri Yükleme:
"""

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_dir = 'dataset/train'
val_dir = 'dataset/val'
test_dir = 'dataset/test'

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

"""#### Modelin Derlenmesi (Compile) ve Callback’lerin Ayarlanması
* EarlyStopping: patience=25
* ReduceLROnPlateau: Validasyon kaybı iyileşmeyince LR’i otomatik düşürecek
"""

# Optimizer seçimi
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=25,
                               restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=5,
                              verbose=1)

# Modelin en iyi epoch'ta kaydedilmesi
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

"""#### Modelin Eğitimi
* Modeli 100 epoch boyunca eğitiyoruz.
"""

epochs = 100
train_steps = train_generator.samples // train_generator.batch_size
val_steps = val_generator.samples // val_generator.batch_size

history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_steps,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

"""# 6. Eğitim/Validasyon Accuracy ve Loss Grafiklerinin Raporlanması
Eğitim süreci boyunca elde edilen kayıp (loss) ve doğruluk (accuracy) değerlerinin grafiklerinin çizilmesi.
"""

num_epochs = min(len(history.history['accuracy']), len(history.history['val_accuracy']))

acc = history.history['accuracy'][:num_epochs]
val_acc = history.history['val_accuracy'][:num_epochs]
loss = history.history['loss'][:num_epochs]
val_loss = history.history['val_loss'][:num_epochs]

epochs_range = range(num_epochs)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

"""## 7. Test Verisi ile Tahmin (Prediction) ve Sonuç Analizi"""

test_generator.reset()
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)

# Tahminlerin en yüksek olasılığa sahip sınıf indeksini bulalım
predicted_classes = np.argmax(predictions, axis=1)

# True classes (flow_from_directory shuffle=False olduğu için indices sıralı)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

"""### 7.1. Confusion Matrix ve Klasik Metrikler"""

cm = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Classification Report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Classification Report:")
print(report)

# Specificity
TP = cm[1,1]
TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]

specificity = TN / (TN + FP)
print("Specificity:", specificity)

"""### 7.2. ROC Eğrisi ve AUC"""

# Gerçek etiketleri 0/1 şeklinde, predicted prob'ları alıyoruz
fpr, tpr, thresholds = roc_curve(true_classes, predictions[:,1])  # sınıf 1 için olasılıklar
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (AUC = %0.4f)' % roc_auc)
plt.plot([0,1], [0,1], color='navy', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

print("AUC: ", roc_auc)

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/omkargurav/face-mask-dataset
License(s): unknown
face-mask-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  face-mask-dataset.zip
replace face_mask_dataset/data/with_mask/with_mask_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 